# Malware Evolution - Test di Ingresso CyberChallenge.it 2021

### Impostazione del problema
    1. Processare l'input e predisposizione delle variabili con cui andremo a lavorare
    3. Per ogni cella della matrice, contare quante delle sue cella adiacenti non sono vuote
    4. Conoscendo il numero di celle adiacenti non vuote, trasformare ogni cella nella sua nuova versione
    5. Ripetere questo procedimento per n volte (round)
    6. Visualizzare a schermo il risultato dei nostri sforzi

### 1. Processare l'input e predisposizione delle variabili con cui andremo a lavorare

In [1]:
#!/bin/python3
from pprint import pprint
from copy import deepcopy

# Opening file descriptor
with open('/home/fabietto/Downloads/evo/input/input1.txt') as f:
    data = f.read() # Dump di tutto il file fino alla fine, compresi i new line
    # data = f.readline() # Dump di una linea del file, si ferma al primo new line
    # data = f.readlines() # Dump di tutte le righe del file, sotto forma di lista

pprint(data)

'10 5 1\n...+.\n**++*\n.*.**\n.+*+*\n*+*+*\n+..++\n+++*.\n*....\n.+..*\n*+***\n'


#### Conversione in lista

In [ ]:
#Conversione data da str a lista (1 elemento per ogni new line)
data = data.strip().split('\n')
pprint(data)

#### Salvataggio delle prime variabili

In [ ]:
rows, columns, rounds_number = map(int,data.pop(0).split(' '))
print(rows, columns, rounds_number)

#### Creazione delle matrice con i dati di input

In [ ]:
matrix = []
for line in  data: 
    temp_list = []
    for c in line:
        temp_list += c
    matrix.append(temp_list)

# Equals to
# for line in data: 
#   matrix.append([c for c in line]) Jupyter Cell Tags support for VS Code


pprint(matrix)

Capiamo un pò come funzionano le coordinate di questa matrice...

In [ ]:
pprint(matrix[0][0]) # Primo elemento della prima riga

pprint(matrix[0][1]) # Secondo elemento della prima riga 

Quindi nelle prime quadra ci vanno le coordinate in y, e nelle seconde ci vanno le coordinate in x, e di conseguenza, per stampare la casella a destra di quella al centro...

In [ ]:
x = 3 
y = 2
pprint(matrix[y][x])

#### Struttura dati dei tipi di celle

Non è un passo strettamente necessario, ma ci aiuterà a non perdere la testa cercando di ricordarci i tipi di celle

In [ ]:
cell_type = {
    'empty'   : '.', 
    'data'    : '+', 
    'malware' : '*'
}

print(cell_type['empty'])
print(cell_type['data'])
print(cell_type['malware'])

### 2. Per ogni cella della matrice, contare quante delle sue cella adiacenti non sono vuote

Come possiamo iterare intorno allo nostra cella?

Se la nostra cella è [y][x], gli offset saranno questi:

```
(y - 1, x - 1)     (y - 1, x + 0)      (y - 1, x + 1)
(y + 0, x - 1)     (y + 0, x + 0)      (y + 0, y + 1)
(y + 1, x - 1)     (y + 1, x + 0)      (y + 1, x + 1)
```

In [ ]:
x = y = 4

non_empty = 0

for offset_y in range(-1, 2): # -1, 0, +1 
    for offset_x in range(-1, 2):

        # Condizioni per non uscire fuori dai limiti delle nostra matrice

        if(x + offset_x > columns - 1):
            print('skipping this cell, too far to the right')
            continue

        if(x + offset_x < 0):
            print('skipping this cell, too far to the left')
            continue

        if(y + offset_y > rows - 1):
            print('skipping this cell, too far down')
            continue

        if(y + offset_y < 0):
            print('skipping this cell, too far up')
            continue

        if(offset_x == 0 and offset_y == 0):
            print('skipping this cell, same element')
            continue

        print(matrix[y + offset_y][x + offset_x])
        
        if(matrix[y + offset_y][x + offset_x] != cell_type['empty']): non_empty += 1

print(non_empty)

### 3. Conoscendo il numero di celle adiacenti non vuote, trasformare ogni cella nella sua nuova versione

Scriviamo una funzione che ci semplifiche la vita

In [ ]:
def transform(cell_content : str, adjacent_non_empty):
    global cell_type
    match cell_content: 
        case '.':
            if(adjacent_non_empty > 4):
                return cell_type['data']
            else:
                return cell_type['empty']
        case '+': 
            if(adjacent_non_empty > 4):
                return cell_type['malware']
            elif(adjacent_non_empty < 4):
                return cell_type['empty']
            else:
                return cell_type['data']
        case '*':
            if(adjacent_non_empty > 4):
                return cell_type['data']
            elif(adjacent_non_empty < 4):
                return cell_type['empty']
            else:
                return cell_type['malware']

### 4. Ripetere questo procedimento per n volte (round)

In [ ]:
for n in range(rounds_number):
    new_matrix = deepcopy(matrix)
    for y in range(rows):
        for x in range(columns):
            print(f"Analyzing {y=}{x=} == {matrix[y][x]}")
            non_empty = []
            for offset_y in range(-1, 2): 
                for offset_x in range(-1, 2):
                    # Condizioni per non uscire fuori dai limiti delle nostra matrice
                    if(x + offset_x > columns - 1):
                        print('skipping this cell, too far to the right')
                        continue
                    if(x + offset_x < 0):
                        print('skipping this cell, too far to the left')
                        continue
                    if(y + offset_y > rows - 1):
                        print('skipping this cell, too far down')
                        continue
                    if(y + offset_y < 0):
                        print('skipping this cell, too far up')
                        continue
                    if(offset_x == 0 and offset_y == 0):
                        print('skipping this cell, same element')
                        continue
                    print(matrix[y + offset_y][x + offset_x])
                    if(matrix[y + offset_y][x + offset_x] != cell_type['empty']): non_empty.append(matrix[y + offset_y][x + offset_x]) 
            print(f"{non_empty=}\nThis cell is now a {transform(matrix[y][x], len(non_empty))}" + '\n' + '*'*10)
            new_matrix[y][x] = transform(matrix[y][x], len(non_empty))
    matrix = new_matrix

pprint(matrix)

### 5. Visualizzare a schermo il risultato dei nostri sforzi

In [ ]:
for row in matrix: 
    for c in row: 
        print(c, end = '')
    print()